In [29]:
import numpy as np
import mne
import pickle

# %matplotlib qt

In [2]:
raw = mne.io.read_raw_edf("inputs\A210223012007.EDF", preload=True)
raw

Extracting EDF parameters from c:\Users\User\workspace\EEG-preprocess\inputs\A210223012007.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 37887  =      0.000 ...   147.996 secs...


<RawEDF | A210223012007.EDF, 22 x 37888 (148.0 s), ~6.4 MB, data loaded>

In [3]:
eeg = raw.copy().pick_types(meg=False, eeg=True, exclude=[])
eeg_indices = mne.pick_channels_regexp(eeg.ch_names, '^EEG')
eeg.info = mne.pick_info(eeg.info, eeg_indices)
eeg.ch_names

['EEG Fp2',
 'EEG Fp1',
 'EEG F4',
 'EEG F3',
 'EEG C4',
 'EEG C3',
 'EEG P4',
 'EEG P3',
 'EEG O2',
 'EEG O1',
 'EEG F8',
 'EEG F7',
 'EEG T4',
 'EEG T3',
 'EEG T6',
 'EEG T5',
 'EEG Fz',
 'EEG Cz',
 'EEG Pz',
 'EEG A2',
 'EEG A1']

In [4]:
map_1020_to_1010 = {
    'T3': 'T7',
    'T4': 'T8',
    'T5': 'P7',
    'T6': 'P8'
}

eeg.rename_channels(lambda s: s.replace("EEG ", ""))
mne.rename_channels(eeg.info, map_1020_to_1010)
geneva_channel_order = ['Fp1', 'F3', 'F7', 'C3', 'T7', 'P3', 'P7', 'O1', 'Pz', 'Fp2', 'Fz', 'F4', 'F8', 'Cz', 'C4', 'T8', 'P4', 'P8', 'O2']
eeg.reorder_channels(geneva_channel_order)
eeg.ch_names

['Fp1',
 'F3',
 'F7',
 'C3',
 'T7',
 'P3',
 'P7',
 'O1',
 'Pz',
 'Fp2',
 'Fz',
 'F4',
 'F8',
 'Cz',
 'C4',
 'T8',
 'P4',
 'P8',
 'O2']

In [5]:
montage = mne.channels.make_standard_montage("standard_1020")
eeg.set_montage(montage, match_case=False)
eeg.ch_names

['Fp1',
 'F3',
 'F7',
 'C3',
 'T7',
 'P3',
 'P7',
 'O1',
 'Pz',
 'Fp2',
 'Fz',
 'F4',
 'F8',
 'Cz',
 'C4',
 'T8',
 'P4',
 'P8',
 'O2']

In [6]:

# eeg_data.plot()
# DEAP Preprocessing

# The data was downsampled to 128Hz.
eeg.resample(128, npad="auto")

# EOG artefacts were removed as in [1].

# A bandpass frequency filter from 4.0-45.0Hz was applied.
eeg.filter(4.0, 45.0)

# The data was averaged to the common reference.
eeg.set_eeg_reference("average")
# The EEG channels were reordered so that they all follow the Geneva order as above.
# Done above

# The data was segmented into 60 second trials and a 3 second pre-trial baseline removed.
# The trials were reordered from presentation order to video (Experiment_id) order.

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 213 samples (1.664 sec)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.


<RawEDF | A210223012007.EDF, 19 x 18944 (148.0 s), ~2.8 MB, data loaded>

In [7]:
eeg.get_data().shape

(19, 18944)

In [8]:
baseline_start = 0
baseline_end = 3

baseline_selection = eeg.copy().crop(tmin=baseline_start, tmax=baseline_end)
baseline_selection

<RawEDF | A210223012007.EDF, 19 x 385 (3.0 s), ~89 kB, data loaded>

In [9]:
eeg_start = 4
eeg_end = 64

eeg_selection = eeg.copy().crop(tmin=eeg_start, tmax=eeg_end)
eeg_selection

<RawEDF | A210223012007.EDF, 19 x 7681 (60.0 s), ~1.1 MB, data loaded>

In [113]:
# eeg_selection.to_data_frame()

In [10]:
eeg_combined = mne.concatenate_raws([baseline_selection, eeg_selection])
eeg_combined

<RawEDF | A210223012007.EDF, 19 x 8066 (63.0 s), ~1.2 MB, data loaded>

In [28]:
eeg_data = eeg_combined.get_data()
eeg_data.shape

(19, 8066)

In [30]:
output = {
    'data': eeg_data
}

with open('test.dat', 'wb') as handle:
    pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
with open('test.dat', 'rb') as handle:
    b = pickle.load(handle)

In [33]:
b['data'].shape

(19, 8066)